# "Predicción de la diferencia de goles y análisis de la ventaja de localía en partidos internacionales de fútbol"
Ciencia de Datos

Profesor: JESUS ANTONIO VILLARRAGA PALOMINO

Integrantes:

Karol Andrea Daza Rojas

Cristian Camilo Berbeo Villalobos

Daniel Felipe Fonseca Cabezas

# "Resumen"
Este cuadernillo documenta el análisis exploratorio, la limpieza y la preparación de datos, la ingeniería de variables, la construcción y evaluación de un modelo de regresión para predecir la diferencia de goles (goal_diff = home_score - away_score) y el análisis de la ventaja de localía usando la base de datos International Football Results (1872–2023) (archivo results.csv). Se incluyen las decisiones de modelado, gráficos y conclusiones.

## "Descripción de la base de datos"
Nombre del archivo: results.csv

Fuente: Conjunto público clásico que registra resultados internacionales desde 1872 hasta 2023 (Kaggle). 

El archivo previsto contiene registros de partidos con las siguientes columnas principales:

### Datos  y Descripcion

|date:        Fecha del partido (YYYY-MM-DD)|

|  home_team: Equipo local                 |

|  away_team: Equipo visitante               |

| home_score: Goles del equipo local            |

| away_score: Goles del equipo visitante          |

| tournament: Tipo de competencia (Friendly, World Cup, etc.)|

|    city: Ciudad donde se jugó             |

|   country: País donde se jugó              |

|   neutral: Booleano que indica si fue en cancha neutral  |


- El dataset es amplio y abarca más de 48.000 partidos.

- Algunas inconsistencias posibles: variaciones en el nombre de torneos, nombres de equipos, y fechas antiguas. Se realizará limpieza y normalización.

## Objetivo General
evaluar un modelo de regresión que prediga la diferencia de goles (goal_diff) en partidos internacionales y cuantifique el efecto de la localía.
### Objetivos Especificos
1. Generar variables derivadas (ingeniería de features) relevantes para la predicción.
2. Entrenar un modelo de regresión (lineal y comparativo con otros) y evaluar su desempeño.
3. Analizar residuales y extraer conclusiones deportivas y estadísticas.

### Estructura de la Base de Datos

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


DATA_PATH = '../data/results.csv'


df = pd.read_csv(DATA_PATH)

df['date'] = pd.to_datetime(df['date'], errors='coerce')

df = df.dropna(subset=['date']).sort_values('date').reset_index(drop=True)


print('Dimensiones:', df.shape)
print(df.columns.tolist())


# Mostrar primeras filas
print(df.head())


### Informacion de la Base de Datos

In [ ]:
print("=== INFORMACIÓN GENERAL DEL DATASET ===")
df.info()

### Resumen Estadistico

In [ ]:
print("=== RESUMEN ESTADÍSTICO ===")
df.describe(include='all')
print(df.describe(include='all'))

### Valores nulos

In [ ]:
print("=== VALORES NULOS POR COLUMNA ===")
df.isnull().sum()

### Distribucion de los goles

In [ ]:
plt.figure(figsize=(7,4))
plt.hist(df['home_score'], bins=30)
plt.title("Distribución de goles del equipo local")
plt.xlabel("Goles")
plt.ylabel("Frecuencia")
plt.show()

plt.figure(figsize=(7,4))
plt.hist(df['away_score'], bins=30)
plt.title("Distribución de goles del equipo visitante")
plt.xlabel("Goles")
plt.ylabel("Frecuencia")
plt.show()
sns.boxplot(x='home_score', data=df)
plt.title("Boxplot de goles del equipo local")

### Cantidad de Goles por año

In [ ]:
df['year'] = df['date'].dt.year

plt.figure(figsize=(12,4))
df['year'].value_counts().sort_index().plot(kind='line')
plt.title("Número de partidos por año")
plt.xlabel("Año")
plt.ylabel("Cantidad de partidos")
plt.show()


### Análisis Exploratorio Inicial

A partir del análisis preliminar de la base de datos pudimos identificar lo siguiente:

- El archivo contiene partidos desde el año 1872 hasta 2023 lo cual significa que teniemos una informacion extensa para el estudio
- Las columnas principales incluyen equipos, goles, ciudad, país y tipo de torneo
- Algunas columnas presentan valores nulos, principalmente en city y country, lo cual se podria esperara debido a que son datos con una gran variedad de antiguedad
- La distribución de goles muestra que la mayoría de partidos tienen entre 0 y 3 goles por equipo
- Se observa un crecimiento significativo en el número de partidos por año, reflejando la expansión del fútbol internacional
- El campo neutral requiere verificación para estandarizarlo (puede venir como booleano o string)
